## Data reading and manipulation

In [2]:
# activate the environment
using Pkg
Pkg.activate("/home/enes/avg/WoR/world_on_rails.jl")

  Activating environment at `~/avg/WoR/world_on_rails.jl/Project.toml`


In [1]:
using YAML: load_file
using Images
using Images.FileIO
using StrLiterals
using LMDB
using LMDB: create, open, close, put!, start

In [3]:
DATA_ROOT = "/home/enes/avg/WoR/dataset"
CONFIG_PATH = "/home/enes/avg/WoR/WorldOnRails/config.yaml"

"/home/enes/avg/WoR/WorldOnRails/config.yaml"

In [31]:
struct MainDataset
    config::Dict
    num_frames::Int
    txn_map::Dict
    idx_map::Dict
    yaw_map::Dict
    file_map::Dict

    function MainDataset(data_dir::AbstractString, config_path::AbstractString)
        # read the config file
        num_frames = 0
        config = load_file(config_path)
        txn_map = Dict()
        idx_map = Dict()
        yaw_map = Dict()
        file_map = Dict()
        
        for full_path in readdir(data_dir, join=true)
            if !isdir(full_path)
                continue
            end
            env = create()
            open(env, full_path)
            txn = start(env)

            dbi = open(txn)
            
            n = parse(Int64, LMDB.get(txn, dbi, "len", String))
            if n < config["num_plan"]+1
                print(full_path, " is too small. consider deleting")
                close(env, dbi)
            else
                offset = num_frames
                for i in 1:n-config["num_plan"]
                    num_frames+=1
                    for j in 1:length(config["camera_yaws"])
                        txn_map[(offset+i)*length(config["camera_yaws"])+j] = txn
                        idx_map[(offset+i)*length(config["camera_yaws"])+j] = i
                        yaw_map[(offset+i)*length(config["camera_yaws"])+j] = j
                        file_map[(offset+i)*length(config["camera_yaws"])+j] = full_path
                    end
                end
            end
        end
        print(data_dir,": ", num_frames, " frames ", "x ", length(config["camera_yaws"]))
        new(config, num_frames, txn_map, idx_map, yaw_map, file_map)
    end
    
    function access(tag, txn, index, T, dtype, preprocess=x->x)
        data = preprocess.([LMDB.get(txn, dbi, f"\(tag)_\(lpad(t, 3, '0')", dtype) for t in index:index+(T-1)])

    end
end

In [32]:
main_dataset = MainDataset(DATA_ROOT, CONFIG_PATH)
lp

/home/enes/avg/WoR/dataset: 195 frames x 3

MainDataset(Dict{Any, Any}("ego_model_dir" => "/kuacc/users/merciyes18/WorldOnRails/ego_model.th", "num_plan" => 5, "camera_yaws" => [0, -30, 30], "multi_cam" => true, "num_orient" => 5, "num_throts" => 3, "all_speeds" => true, "temperature" => 0.01, "ego_data_dir" => ["PATH TO PHASE 0 DATA"], "min_speeds" => 0.0…), 195, Dict{Any, Any}(56 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 35 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 425 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 429 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 60 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 220 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 308 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 67 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 215 => Transaction(Ptr{Nothing} @0x000000000398a7f0), 73 => Transaction(Ptr{Nothing} @0x000000000398a7f0)…), Dict{Any, Any}(56 => 18, 35 => 11, 425 => 141, 429 => 142, 60 => 19, 220 => 73, 308 => 102, 67 => 22, 215 => 71, 73 => 24…), 

In [35]:
Base.length(d::MainDataset) = (if (d.config["multi_cam"]) d.num_frames*length(d.config["camera_yaws"]) else d.num_frames end)

In [36]:
length(main_dataset)

585

In [ ]:
function get_item(d::MainDataset, state::Int)
    idx = state
    if !d.config["multi_cam"]
        idx *= length(d.config["camera_yaws"])
    end
    
    lmdb_txn = d.txn_map[idx]
    index = d.idx_map[idx]
    cam_index = d.yaw_map[idx]

    
end

In [37]:
Base.iterate(d::MainDataset, state=1) = state > length(d) ? nothing : get_item(d, state), state+1 

In [40]:
close(env)

LoadError: UndefVarError: env not defined